In [1]:
#Defining Libraries:
from bitcoin import *
import base58
import os
import hashlib
import ecdsa
import binascii
from ecdsa import SigningKey, SECP256k1
from prettytable import PrettyTable

## Address generation

In [2]:
#Defining Function For Hashing: 1. Sha256 and 2. Ripemd160
def hashing(a):
    first_sha256 = hashlib.sha256(a)
    ripemd160 = hashlib.new("ripemd160")
    ripemd160.update(first_sha256.digest())
    return_0 = ripemd160.digest()
    return_1 = bytes.fromhex("00") + ripemd160.digest()
    return_2 = bytes.fromhex("6f") + ripemd160.digest()
    return return_0, return_1, return_2

#Function for Checksum
def checksum(b):
    checksum_full = hashlib.sha256(hashlib.sha256(b).digest()).digest()
    new_checksum = checksum_full[:4]
    return b + new_checksum

#Function to convert to base58    
def to_base58(c):
    return base58.b58encode(c).decode('utf-8')

def compressed_key(d):
    a = d.hex()

    if a[-1] == '0' or a[-1] == '2' or a[-1] == '4' or a[-1] == '6' or a[-1] == '8' or a[-1] == 'a' or a[-1] == 'c' or a[-1] == 'e':
        return bytes.fromhex("02") + d
    
    else:
        return bytes.fromhex("03") + d
    
def generate_address(print_output = True):
    
    #generating Random number of 32 bytes
    private_key = os.urandom(32).hex()
    private_key_from_hex = bytes.fromhex(private_key)

    #Keeping same private key, Generating Public Key
    import ecdsa # pip install ecdsa
    sk = ecdsa.SigningKey.from_string(private_key_from_hex, curve = ecdsa.SECP256k1) 


    #actual publickey to verify messages that singed using own pvt key
    verification_key = sk.verifying_key
    #making Public Key:
    public_key = bytes.fromhex("04") + verification_key.to_string()
    compressed_public_key = compressed_key(verification_key.to_string())
    compressed_public_key_hex = compressed_public_key.hex()[:66]
    #Converting public key to address:
    decoded_pubkey, mainnet_pubkey, testnet_pubkey = hashing(public_key)

    #DECODED PUBLIC KEY WILL BE REQUIRED
    #checksum:
    checksum_main_pubkey = checksum(mainnet_pubkey)
    checksum_test_pubkey = checksum(testnet_pubkey)


    main_address = to_base58(checksum_main_pubkey)
    test_address = to_base58(checksum_test_pubkey)


    if print_output:
        x = PrettyTable()
        x.field_names = ["Description", "Value"]
        x.add_row(["Private Key, hex", private_key])
        x.add_row(["Public Key, hex", public_key.hex()])
        x.add_row(["Compressed Public Key, hex", compressed_public_key_hex])
        x.add_row(["MainNet Address, base58", main_address])
        x.add_row(["Testnet Address, base58", test_address])
        print(x)
    return(private_key, public_key.hex(), compressed_public_key_hex, main_address, test_address)

In [3]:
print('''
===============================
First address data
================================''')
address_data_1 = generate_address()
print('''
===============================
Second address data
================================''')
address_data_2 = generate_address()
print('''
===============================
Third address data
================================''')
address_data_3 = generate_address()


First address data
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|        Description         |                                                               Value                                                                |
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|      Private Key, hex      |                                  39e154ce54e42e3a0efbe5bf49f0182a4daabe1e0318dbf40eda24ab61685721                                  |
|      Public Key, hex       | 04a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f3 |
| Compressed Public Key, hex |                                 03a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572e                               

In [4]:
def public_to_test(scriptaddress):
    decoded_address = base58.b58decode(scriptaddress)
    address_1 = decoded_address[1:-4]
    hex_address = bytes.fromhex("C4") + address_1
    checksum_test_multsig = checksum(hex_address)
    test_address = to_base58(checksum_test_multsig)
    return test_address

## 2-3 MultiSignature address generation

In [5]:
rawscript = mk_multisig_script([
    '04a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f3', 
    '04bc14c4d298518f695537cafe06666e66ea163241ee2c999f4aa5dae589c797c42db3d42d3ce5c2adf18dfb1b6835a4d943d163671443afe070a6dd6e96c55cfd',
    '04fc5799bef2287d23e93afa3a2764016961540101b24eaa483ee8374a4d988174a6beb4c4aeb57e90c2a50759e4b02ddcc3a49679a36aa6d32239a00881a7d389'],
    2, 3)
descript = deserialize_script(rawscript)
scriptaddress = scriptaddr(rawscript)

testnet_address = public_to_test(scriptaddress)
print("Multisig testnet address: " + testnet_address)

Multisig testnet address: 2N93nThjGPWVevvMuKF7t8KHBHLxfqxy4sS


### №1 (1 point) 
**Task**:

Calculate double Hash_256 of your name. Take two inputs from the user, i.e., first name and last name. Concatenate string and take SHA256(SHA256(<first name><lastname>))

In [1]:
def doubleSHA256(data):    
    hash = hashlib.sha256(data).digest()     
    return hashlib.sha256(hash).digest()

In [7]:
name = input("Enter your first name: ")
surname = input("Enter your second name: ")
concat_str = name + surname
print("Concatenated string:\n      " + concat_str)
toHash = concat_str.encode('utf-8')
doublehashed = doubleSHA256(toHash)
print("Double-SHA256:\n      " + binascii.hexlify(doublehashed).decode('utf8'))

Enter your first name: Inna
Enter your second name: Tarasyuk
Concatenated string:
      InnaTarasyuk
Double-SHA256:
      0002600c78e4f6339c8e135e3b8ec430b097dc13a4bc44c37b363afe09dfd532


### №2 (1 point)
**Task**:

Get some testnet Bitcoins to your testnet address_1 from any source. 

(For example: https://testnet-faucet.mempool.co/, https://coinfaucet.eu/en/btc-testnet/ or search "bitcoin testnet faucet"). 

Provide the Transaction_ID.

**Solution**:

**Transaction ID**: 11879ad2ef3994fbc2df1113558038b7df6f70c1daafb9b92261369d8c484d94

**Link to the transaction**: https://blockstream.info/testnet/tx/11879ad2ef3994fbc2df1113558038b7df6f70c1daafb9b92261369d8c484d94?expand

### №3 (2 points) 

**Task**:

Create a new transaction from address_1 and provide Transaction_ID. 

Make 2 outputs:

        Output_1 : Some bitcoins to address_2.

        Output_2: Data transfer with first 4 Bytes double Hash_256 of your name.

You send 0 bitcoins in this output.

Note. Be aware of small transaction fees. You may find https://blockstream.info/testnet/tx/push useful.

**Solution**:

Txid: 11879ad2ef3994fbc2df1113558038b7df6f70c1daafb9b92261369d8c484d94

Ouput index: 1

Previous pkh: c9d3639a088c1dc9eb0bd0b51663266ac988e0cb

Receiver address: myLzysoAaGvpHvmACK6qkRX67CPVcKjkeD

Data transfer: 0002600c

Private key: 39e154ce54e42e3a0efbe5bf49f0182a4daabe1e0318dbf40eda24ab61685721

Public key: 04a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f3

In [8]:
class variableInteger:
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            #print(a[i] + a[i+1])
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)
    
    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b
    
    def variable_integer(a):
        if type(a) == int:
            temp = a
        else:
            temp = int(len(a)/2)
        
        if temp <= int('0xfc',16):
            return variableInteger.padding(hex(temp)[2:])
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))


def bytes_padding(temp_no):
    if len(temp_no)/2 != 0:
        return '0' + temp_no
    else:
        return temp_no
    
        
def op_return():
    value_return = '0'*16
    #data_1 = b"charley loves heidi"
    data = input("Enter Data to send, for exampe, 'Hello': ")
    text_from_bytes = binascii.b2a_hex(data.encode("utf-8")).decode()
    len_data = hex(int(len(text_from_bytes)/2))[2:]
    partial_data =  '6a'+ bytes_padding(len_data) + text_from_bytes
    return value_return + bytes_padding(hex(int(len(partial_data)/2))[2:]) + partial_data
        
def reverse(a):
    i = 0
    rev_a = []
    while i != len(a):
        temp_value = a[i] + a[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def tx_outputs(c = 0):
    receiver = input("Enter Receiver's Address {}: ".format(c+1))
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = '76' + 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '88' + 'ac'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script

def bytewise_reverse(hex_string):
    i = 0
    reverted_string = []
    while i != len(hex_string):
        temp_value = hex_string[i] + hex_string[i+1]
        reverted_string.append(temp_value)
        i = i + 2
    reverted_string.reverse()
    return ''.join(reverted_string)

def doublehash256_txid(a): 
    temp_bin = binascii.unhexlify(a)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).digest()
    return str(binascii.hexlify(hash2),"ascii")
    
def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = bytewise_reverse(str(temp_1).zfill(16))
    return temp_2

    
def pushdata(len_r,len_s):
    if len_r == '20' and len_s == '20':
        return str(47)    
    elif len_r == '20' or len_s == '21':
        return str(48)
    elif len_r == '21' or len_s == '20':
        return str(48)
    else:
        return str(49)
    
def pubkey_opcode(a):
    return hex(int(int(len(a))/2))[2:]


version_number = '01000000'
number_inputs = '01'
previous_transaction = input("Enter user transaction: ")
previous_transaction = reverse(previous_transaction)
output_script = '0' + input("Enter previous output (here <= 15): ") + '000000'
script = '1976a914' + input("Enter previous PKH: ") + '88ac'
sequence = 'ffffffff'
tx_ip = version_number + number_inputs + previous_transaction + output_script + script + sequence


#User_output:
Outputs = '02'
out_tx = Outputs + tx_outputs() + op_return() + '0'*8 + '01000000'

#Final Transaction:
raw_transaction = tx_ip + out_tx
print('''Raw transaction: ''', raw_transaction)

#DIGITAL SIGNATURE
unsigned = raw_transaction

temp_bin = binascii.unhexlify(unsigned)
temp_hash = hashlib.sha256(temp_bin).digest()
hash2 = hashlib.sha256(temp_hash).hexdigest()
tx_hash = hashlib.sha256(temp_hash).digest()

private_key = input("\nEnter Private Key: ")
signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)

print("\n\nDigital Signature: ", str(binascii.hexlify(SIG),'ascii'))

#Generating_SigScript:
DER_input =  str(binascii.hexlify(SIG),'ascii')
header = DER_input[0:2]
Sig_Length = DER_input[2:4]
r_integer = DER_input[4:6]
r_length = DER_input[6:8]
r = DER_input[8:8 + 2 * int(r_length,16)]
s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
s = DER_input[12 + 2 * int(r_length,16):]

pushdata_opcode = pushdata(r_length, s_length)
sighash_code = '01'

publickey = input("Enter Publickey (Compressed or Uncompressed): ")
pubkey_pushdata_opcode = pubkey_opcode(publickey)

final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
SigScript = pubkey_opcode(final_SigScript) + final_SigScript
print("\nSigScript(hex): ", SigScript,"\n\n\n")

#Broadcasting_Transaction:
broadcasting_tx = version_number + number_inputs + previous_transaction + output_script + SigScript + sequence + out_tx [:-8]
print('''Transaction to broadcast: ''', broadcasting_tx)

Enter user transaction: 11879ad2ef3994fbc2df1113558038b7df6f70c1daafb9b92261369d8c484d94
Enter previous output (here <= 15): 1
Enter previous PKH: c9d3639a088c1dc9eb0bd0b51663266ac988e0cb
Enter Receiver's Address 1: myLzysoAaGvpHvmACK6qkRX67CPVcKjkeD
Enter Value in BTC (0.0) to send myLzysoAaGvpHvmACK6qkRX67CPVcKjkeD:0.015
Enter Data to send, for exampe, 'Hello': 0002600c
Raw transaction:  0100000001944d488c9d366122b9b9afdac1706fdfb73880551311dfc2fb9439efd29a8711010000001976a914c9d3639a088c1dc9eb0bd0b51663266ac988e0cb88acffffffff0260e31600000000001976a914c39058dead7c1f402a76fb046e415f135785820a88ac00000000000000000a6a0830303032363030630000000001000000

Enter Private Key: 39e154ce54e42e3a0efbe5bf49f0182a4daabe1e0318dbf40eda24ab61685721


Digital Signature:  30450221008f9785f676fb6d429394a4ac1454295f52653463039c670eb669a8610ac50669022068a325e2d324f06837d9456946576905943383f617a45a507b71833f45a46d5f
Enter Publickey (Compressed or Uncompressed): 04a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0

**Transaction ID**: 1e43c2dadd115a7bf0a8ba928e79329c31cdfc21d0f86838d5dbaaee2a2b9010

**Link to the transaction**: https://blockstream.info/testnet/tx/1e43c2dadd115a7bf0a8ba928e79329c31cdfc21d0f86838d5dbaaee2a2b9010?expand

### №4 (2 points) 

**Task**:

From address_2 send some bitcoins to your 2-of-3 MultiSignature Address.

In [10]:
def tx_inputs_tx_id():
    previous_tx_hash = input("Enter Reference TxId: ")
    previous_tx_hash_reversed = bytewise_reverse(previous_tx_hash)
    previous_output_index = input("Enter Previous output index: ")
    previous_output_index = bytewise_reverse('{:08x}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index + place_holder() +  'ffffffff'


def place_holder():
    previous_place_holder = input('''Enter Output Script: 
(here: 160 bit result of OP_HASH160(*)=RIPEMD160(SHA256(*)): ''')
    size_receiver_1 = hex(int(len(previous_place_holder)/2))[2:]
    script_Pubkey_prev_op = '76' + 'a9' + size_receiver_1 + previous_place_holder + '88' + 'ac'
    script_len_1 = hex(int(len(script_Pubkey_prev_op)/2))[2:]
    return script_len_1 + script_Pubkey_prev_op

def generate_raw_transaction(print_output = True):
    #Creating Raw Transaction for Double Hash
    #P2PKH
    version = 1
    version = bytewise_reverse('{:08d}'.format(int(version)))

    #_________________TALKING PREVIOUS TRANSACTIONS_________________
    utxo_ip = '01'
    prev_tx_datas = utxo_ip + tx_inputs_tx_id()


    #___________________________________________________________________
    utxo_output = input("Enter number of outputs: ")
    tx_output = ''
    for i in range(0, int(utxo_output)):
            tx_output = tx_outputs(i) + tx_output
    next_output_data = '0' + utxo_output + tx_output
    #____________________________________________________________________________

    locktime = '00000000'
    SigHashCode = '01000000'


    #Creating Raw transaction:
    print(
    """
    Raw_Transaction for 1 input Data and {} Output
    """.format(tx_output))
    #Printing Input PreSignature:

    raw_transaction = version + prev_tx_datas + next_output_data + locktime + SigHashCode
    if print_output:
        print(raw_transaction)
    return(raw_transaction)

def sign_raw_transaction(print_output = True):
    unsigned = input("Enter Raw data Transaction: ")

    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).hexdigest()
    tx_hash = hashlib.sha256(temp_hash).digest()

    private_key = input("Enter Private Key: ")
    signingKey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingKey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)
    SIG = str(binascii.hexlify(SIG),'ascii')
    if print_output:
        print("\n\nSig Script: ", SIG)
    return(SIG)

def decode_digital_signature():
    #_______Digital Signature Decoder__________
    DER_input =  input("Enter Digital Signature: ")
    print("\n")

    #Variables:
    header = DER_input[0:2]
    Sig_Length = DER_input[2:4]
    r_integer = DER_input[4:6]
    r_length = DER_input[6:8]
    r = DER_input[8:8 + 2 * int(r_length,16)]
    s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
    s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
    s = DER_input[12 + 2 * int(r_length,16):]

    print("\033[4mDECODING DIGITAL SIGNATURE\033[0m\n")
    x = PrettyTable()
    x.field_names = ["Fields","Value (in Hex)"]
    x.add_row(["Header",header])
    x.add_row(["Sig Length",Sig_Length])
    x.add_row(["Integer",r_integer])
    x.add_row(["R Length",r_length])
    x.add_row(["R",r])
    x.add_row(["Integer",s_integer])
    x.add_row(["S Length",s_length])
    x.add_row(["S",s])
    print(x)


    #_____DER_SIG_SCRIPT_ENCODER____________
    def pushdata(len_r, len_s):
        if len_r == '20' and len_s == '20':
            return str(47)    
        elif len_r == '20' or len_s == '21':
            return str(48)
        elif len_r == '21' or len_s == '20':
            return str(48)
        else:
            return str(49)

    def pubkey_opcode(a):
        return hex(int(int(len(a))/2))[2:]



    pushdata_opcode = pushdata(r_length, s_length)
    sighash_code = '01'

    publickey = input("Enter Publickey (Compressed or Uncompressed) : ")
    pubkey_pushdata_opcode = pubkey_opcode(publickey)

    y = PrettyTable()

    print("\033[4mSIG SCRIPT\033[0m\n")
    y.field_names = ["Fields","Value (in Hex)"]
    y.add_row(["PUSHDATA Opcode",pushdata_opcode])
    y.add_row(["Header",header])
    y.add_row(["Sig Length",Sig_Length])
    y.add_row(["Integer",r_integer])
    y.add_row(["R Length",r_length])
    y.add_row(["R",r])
    y.add_row(["Integer",s_integer])
    y.add_row(["S Length",s_length])
    y.add_row(["S",s])
    y.add_row(["------","------"])
    y.add_row(["SigHash Code",sighash_code])
    y.add_row(["PUSHDATA Opcode",pubkey_pushdata_opcode])
    y.add_row(["Public Key",publickey])

    print(y)

    final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
    z = PrettyTable()
    z.field_names = ["Fields","Value (in Hex)"]
    z.add_row(["SigScript Lenght",pushdata_opcode])
    z.add_row(["Final SigScript",final_SigScript])

    print(z)
    print("\nSigScript(hex): ",pubkey_opcode(final_SigScript) + final_SigScript,"\n\n\n")

**Solution**:

Reference txid: 1e43c2dadd115a7bf0a8ba928e79329c31cdfc21d0f86838d5dbaaee2a2b9010

Previous output index: 0

Previous output script: c39058dead7c1f402a76fb046e415f135785820a

Number of outputs: 1

Receiver's address: 2N93nThjGPWVevvMuKF7t8KHBHLxfqxy4sS

Value in BTC: 0.012

Address2 private key: 962f5c4ccf959e70cb391a7d02b80eaf2487d6282e41493d0b9df1b012f7a33f

Address2 public key: 04bc14c4d298518f695537cafe06666e66ea163241ee2c999f4aa5dae589c797c42db3d42d3ce5c2adf18dfb1b6835a4d943d163671443afe070a6dd6e96c55cfd

In [11]:
generate_raw_transaction()
sign_raw_transaction()
decode_digital_signature()

Enter Reference TxId: 1e43c2dadd115a7bf0a8ba928e79329c31cdfc21d0f86838d5dbaaee2a2b9010
Enter Previous output index: 0
Enter Output Script: 
(here: 160 bit result of OP_HASH160(*)=RIPEMD160(SHA256(*)): c39058dead7c1f402a76fb046e415f135785820a
Enter number of outputs: 1
Enter Receiver's Address 1: 2N93nThjGPWVevvMuKF7t8KHBHLxfqxy4sS
Enter Value in BTC (0.0) to send 2N93nThjGPWVevvMuKF7t8KHBHLxfqxy4sS:0.012

    Raw_Transaction for 1 input Data and 804f12000000000017a914ad5820bcb7650a8bb716a8b006ba8f6f3af8738387 Output
    
010000000110902b2aeeaadbd53868f8d021fccd319c32798e92baa8f07b5a11dddac2431e000000001976a914c39058dead7c1f402a76fb046e415f135785820a88acffffffff01804f12000000000017a914ad5820bcb7650a8bb716a8b006ba8f6f3af87383870000000001000000
Enter Raw data Transaction: 010000000110902b2aeeaadbd53868f8d021fccd319c32798e92baa8f07b5a11dddac2431e000000001976a914c39058dead7c1f402a76fb046e415f135785820a88acffffffff01804f12000000000017a914ad5820bcb7650a8bb716a8b006ba8f6f3af8738387000000000100

tx_unsigned = 010000000110902b2aeeaadbd53868f8d021fccd319c32798e92baa8f07b5a11dddac2431e000000001976a914c39058dead7c1f402a76fb046e415f135785820a88acffffffff01804f12000000000017a914ad5820bcb7650a8bb716a8b006ba8f6f3af87383870000000001000000

tx_unsigned2 = 010000000110902b2aeeaadbd53868f8d021fccd319c32798e92baa8f07b5a11dddac2431e000000001976a914c39058dead7c1f402a76fb046e415f135785820a88acffffffff01804f12000000000017a914ad5820bcb7650a8bb716a8b006ba8f6f3af873838700000000

tx_final = 010000000110902b2aeeaadbd53868f8d021fccd319c32798e92baa8f07b5a11dddac2431e000000008a47304402203028986a8577d1722a0868e273503b0881cfafdad26f8032abf27a8452055e5a02203bafbbd870c3f9550b129a3f19bc99c9e4e951506a7217f4f7e2ba1d28586a14014104bc14c4d298518f695537cafe06666e66ea163241ee2c999f4aa5dae589c797c42db3d42d3ce5c2adf18dfb1b6835a4d943d163671443afe070a6dd6e96c55cfdffffffff01804f12000000000017a914ad5820bcb7650a8bb716a8b006ba8f6f3af873838700000000

**Transaction ID**: 510b3746d2b65358251431cfccf8f87c302342367c1cfdfcbdafe3f0ec7b305a

**Link to the transaction**: https://blockstream.info/testnet/tx/510b3746d2b65358251431cfccf8f87c302342367c1cfdfcbdafe3f0ec7b305a?expand

### №5 (4 points) 

**Task**:

From 2-of-3 MultiSignature Address send some bitcoins back to address_1

**Solution**:

Redeem script:

In [12]:
rawscript

'524104a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f34104bc14c4d298518f695537cafe06666e66ea163241ee2c999f4aa5dae589c797c42db3d42d3ce5c2adf18dfb1b6835a4d943d163671443afe070a6dd6e96c55cfd4104fc5799bef2287d23e93afa3a2764016961540101b24eaa483ee8374a4d988174a6beb4c4aeb57e90c2a50759e4b02ddcc3a49679a36aa6d32239a00881a7d38953ae'

Reference txid: 510b3746d2b65358251431cfccf8f87c302342367c1cfdfcbdafe3f0ec7b305a

Previous output index: 0

Redeem script: 524104a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f34104bc14c4d298518f695537cafe06666e66ea163241ee2c999f4aa5dae589c797c42db3d42d3ce5c2adf18dfb1b6835a4d943d163671443afe070a6dd6e96c55cfd4104fc5799bef2287d23e93afa3a2764016961540101b24eaa483ee8374a4d988174a6beb4c4aeb57e90c2a50759e4b02ddcc3a49679a36aa6d32239a00881a7d38953ae

Receiver's address: myv7MBWKrBqD814SqfwCGB6xeVRJTnFHeS

Address1 private key: 39e154ce54e42e3a0efbe5bf49f0182a4daabe1e0318dbf40eda24ab61685721

Address3 private key: 1046ce4f12c6daec23e99d2faffc19e9084977ea3e972bfb1ae1056624757926

In [13]:
# code from seminar
class variableInteger:
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            #print(a[i] + a[i+1])
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)
    
    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b
    
    def variable_integer(a):
        if type(a) == int:
            temp = a
        else:
            temp = int(len(a)/2)
        
        if temp <= int('0xfc',16):
            return variableInteger.padding(hex(temp)[2:])
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        
def digital_signature(raw_tx):
    unsigned = raw_tx
    
    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    tx_hash = hashlib.sha256(temp_hash).digest()
    private_key = input("Enter Private Key: ")
    signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)   
    temp_sig = str(binascii.hexlify(SIG),'ascii')
    return temp_sig     

def reverse_string_in_pair(a_str_ip):
    i = 0
    rev_a = []
    while i != len(a_str_ip):
        #print(a[i] + a[i+1])
        temp_value = a_str_ip[i] + a_str_ip[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = reverse_string_in_pair(str(temp_1).zfill(16))
    return temp_2

def tx_inputs_tx_id():
    previous_tx_hash = input("Enter Reference TxId: ")
    previous_tx_hash_reversed = reverse_string_in_pair(previous_tx_hash)
    previous_output_index = input("Enter Previous output index: ")
    previous_output_index = reverse_string_in_pair('{:08d}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index

def redeem_place_holder():
    redeemScript = input("Enter redeem Script: ")
    return variableInteger.variable_integer(redeemScript) + redeemScript

def tx_outputs(c):
    receiver = input("Enter Receiver's Address {}: ".format(c+1))
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = '76' + 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '88' + 'ac'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script

sig_loop = input("Number of Signatures Required: ")
#Making Raw_Transaction: 
#input Basic
version = '01000000'

#input_multi_Signature:
input_ms = '01' + tx_inputs_tx_id()
redeem_Script = redeem_place_holder()
tx_input_value = input_ms + redeem_Script + 'f'*8

#output
utxo_output = input("Enter number of outputs: ")
tx_output = ''
for i in range(0, int(utxo_output)):
        tx_output = tx_outputs(i) + tx_output
next_output_data =  variableInteger.variable_integer(int(utxo_output)) + tx_output

locktime = '00000000'
SigHashCode = '01000000'

tx_to_sign = version + tx_input_value + next_output_data + locktime + SigHashCode
print("Raw Transaction to Sign:")
print(tx_to_sign)

signatures = ''

for i in range(0,int(sig_loop)):
    signature_temp = digital_signature(tx_to_sign) + '01'
    signature_temp_2 = variableInteger.variable_integer(signature_temp) + signature_temp
    signatures = signatures + signature_temp_2
    print(i+1,signatures)
    
SigScript_places = '00' + signatures + '4c' + redeem_Script 
SigScript =  variableInteger.variable_integer(SigScript_places) + SigScript_places
print("SigScript:",SigScript)

Broadcasting_transaction = version + input_ms + SigScript + 'f'*8 + next_output_data + locktime
print(Broadcasting_transaction)

Number of Signatures Required: 2
Enter Reference TxId: 510b3746d2b65358251431cfccf8f87c302342367c1cfdfcbdafe3f0ec7b305a
Enter Previous output index: 0
Enter redeem Script: 524104a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f34104bc14c4d298518f695537cafe06666e66ea163241ee2c999f4aa5dae589c797c42db3d42d3ce5c2adf18dfb1b6835a4d943d163671443afe070a6dd6e96c55cfd4104fc5799bef2287d23e93afa3a2764016961540101b24eaa483ee8374a4d988174a6beb4c4aeb57e90c2a50759e4b02ddcc3a49679a36aa6d32239a00881a7d38953ae
Enter number of outputs: 1
Enter Receiver's Address 1: myv7MBWKrBqD814SqfwCGB6xeVRJTnFHeS
Enter Value in BTC (0.0) to send myv7MBWKrBqD814SqfwCGB6xeVRJTnFHeS:0.01
Raw Transaction to Sign:
01000000015a307becf0e3afbdfcfd1c7c364223307cf8f8cccf3114255853b6d246370b5100000000c9524104a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f34104bc14c4d298518f6

Transaction to broadcast: 
01000000015a307becf0e3afbdfcfd1c7c364223307cf8f8cccf3114255853b6d246370b5100000000fd5c01004730440220424fe0d9ba404d8bef5378aa306e22fbfb48d1e8f5a1fe61d2f616cca3df865202207e7df622c65e5a32ac92b6a294ff04e1d2f84ae210fff0ac92092349ded15c0e014730440220346cffc912d34608f6686b82ac06286ffd5c339421a53cb00c38030f38cc8d7902204cdf18122fff43f5b29980f929e38b475753c14894d003f479ad93f49d6de23e014cc9524104a9decc7b52adc5c97aaf8a02b35b6dd71d562d69a0ed69ad594a12f5d463572ed45b2c8d66020c21ced69273a82675d60dd46d5a1eb019890a7dc64c10ef75f34104bc14c4d298518f695537cafe06666e66ea163241ee2c999f4aa5dae589c797c42db3d42d3ce5c2adf18dfb1b6835a4d943d163671443afe070a6dd6e96c55cfd4104fc5799bef2287d23e93afa3a2764016961540101b24eaa483ee8374a4d988174a6beb4c4aeb57e90c2a50759e4b02ddcc3a49679a36aa6d32239a00881a7d38953aeffffffff0140420f00000000001976a914c9d3639a088c1dc9eb0bd0b51663266ac988e0cb88ac00000000

**Transaction ID**: e07047daa95f484c2ce9e935494decb22fdb4d07ae44c905ea05aa501a1ae5cc

**Link to the transaction**: https://blockstream.info/testnet/tx/e07047daa95f484c2ce9e935494decb22fdb4d07ae44c905ea05aa501a1ae5cc?expand